In [2]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

## 1) Tor-Selenium Setup

In [4]:
torexe = os.popen(r'C:\Users\Daniel\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')

firefoxBinary = r"C:\Users\Daniel\Desktop\Tor Browser\Browser\firefox.exe"
geckodriverPath = r"C:\Users\Daniel\Desktop\Tor Browser\geckodriver.exe"
proxyIP = "127.0.0.1"
proxyPort = 9150

binary = FirefoxBinary(firefoxBinary)

proxy_settings = {
    "network.proxy.type": 1,
    "network.proxy.socks": proxyIP,
    "network.proxy.socks_port": proxyPort,
    "network.proxy.socks_remote_dns": False,
    "extensions.torlauncher.start_tor": True
}

profile = FirefoxProfile(r'C:\Users\Daniel\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
profile.set_preference('extensions.torlauncher.start_tor', True) # bypass tor connection page
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", True)
profile.update_preferences()

driver = webdriver.Firefox(executable_path=geckodriverPath, 
                           firefox_binary=binary, 
                           firefox_profile=profile)



## 2) Scraping Medusa

Obtain all information about threatened companies posted in Medusa Blog  
- Company Name  
- Company Description  
- Date Published by Medusa  
- Number of clicks  
- Type of data stolen (not sure about how to extract this)

BlackMatter link: http://medusaxko7jxtrojdkxo66j7ck4q5tgktf7uqsqyfry4ebnxlcbkccyd.onion/

Medusa Blog is displayed in a timeline format where content is continually loaded as you scroll down to the end of the webpage. This means we need to scroll all the way down, then extract the information.  

To know when the webpage is fully loaded, either scroll all the way down each iteration until there is no new content loaded after an explicit wait, or search for the absence of HTML element "progress" after scrolling all the way down each iteration.  

Each company information is in the form of the HTML element "Card" and have the class "card"  

In [5]:
# Wait till your tor browser is connected to the tor network before executing this
driver.get('http://medusaxko7jxtrojdkxo66j7ck4q5tgktf7uqsqyfry4ebnxlcbkccyd.onion/')

In [6]:
SCROLL_PAUSE_TIME = 5 # as the Tor browser is slow, might need to increase this accordingly

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [7]:
companyNames = []
companyDescription = []
dateVictimised = []
noClicks = []

cards = driver.find_elements_by_xpath("//div[@class='card']")

for card in cards:
    dateVictimisedElement = card.find_element_by_xpath(".//div[@class='date-updated']/span[@class='text-muted']")
    dateVictimisedText = dateVictimisedElement.text
    # if dateVictimisedText[0:4] != "2023":
    #     continue
    dateVictimised.append(dateVictimisedText)

    companyNameElement = card.find_element_by_xpath(".//h3[@class='card-title']")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)

    companyDescriptionElement = card.find_element_by_xpath(".//p[@class='card-text text-left']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)

    noClicksElement = card.find_element_by_xpath(".//div[@class='number-view']/span[@class='text-muted']")
    noClicksText = noClicksElement.text
    noClicks.append(noClicksText)

driver.quit()


## 3) Clean data + Extract to CSV/Excel format


Requirements for data:  
- Date of victim company within 2023

In [8]:
df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, "Date Victimised": dateVictimised, "Number of Clicks": noClicks})
df['Date Victimised'] = pd.to_datetime(df['Date Victimised'], format='%Y-%m-%d %H:%M:%S')

date_mask = df['Date Victimised'].dt.year < 2023

df = df[~date_mask]

df.reset_index(drop=True)

df

,Company,Company Description,Date Victimised,Number of Clicks
0,Somagic,The Somagic is a French company in the product...,2023-10-03 14:45:58,337
1,Windak,Windak is a cable packaging equipment manufact...,2023-10-02 08:19:08,651
2,Karam Chand Thapar & Bros Coal Sales,Karam Chand Thapar & Bros Coal Sales - it is t...,2023-10-02 07:57:12,623
3,LANDSTAR POWER ONTARIO INC,LANDSTAR POWER ONTARIO INC - canadian company ...,2023-09-26 23:14:54,2070
4,Acoustic Center,Acoustic Center is an Italian regional company...,2023-09-25 23:12:15,1959
...,...,...,...,...
96,European Window,European Window Company is an Australian leade...,2023-02-02 18:24:06,10292
97,EightPixelsSquare,"Founded in December 2012 and based in Derby, U...",2023-01-25 18:01:43,11011
98,Aglobis,Resistance shall not prevent us from fulfillin...,2023-01-20 17:59:08,10374
99,Integerity Tax,"Small business accounting, tax preparation, bo...",2023-01-11 13:11:48,9928


In [8]:
df.to_csv("Medusa_Data.csv")

## 3) Scraping Glassdoor for more company information

More information we need:  
- Industry  
- Geography  
  - Country  

Additional information we can scrape:
- Company Website

In [9]:
# pip install webdriver-manager
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [39]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()

In [40]:
import requests

def location_to_country(location):
    base_url = "https://nominatim.openstreetmap.org/search"
    
    params = {
        "addressdetails": 1,
        "q": location,
        "format": "jsonv2",
        "limit": 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data:
        country = data[0].get("address", {}).get("country", "")
        return country

    return np.nan

In [41]:
# Using LinkedIn

companyWebsite_list = []
industry_list = []
headquarter_list = []
country_list = []
number = 0

wait = WebDriverWait(driver, 5)

for company in companyNames: 
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(company + ' linkedin')
        search.send_keys(Keys.RETURN)
        linkedin_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        time.sleep(2)
        linkedin_page.click()

        if "linkedin" in driver.current_url:

            loaded = wait.until(EC.presence_of_element_located((By.ID, "main-content")))

            # Check if the scraped company name is even in the linkedin page (sometimes the company data is not in linkedin; false positive)
            # try:
            #     linkedin_companyName_element = driver.find_element_by_xpath("//h1[@class='top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0']")
            #     linkedin_companyName = linkedin_companyName_element.text
            # except:
            #     print(company)
            #     continue

            # if not substring_included(company, linkedin_companyName):
            #     print(company)
            #     continue

            CLASS = ".//dd[@class='font-sans text-md text-color-text break-words overflow-hidden']"

            try:
                website_element = driver.find_element_by_css_selector('[data-test-id="about-us__website"]')
                website_text_element = website_element.find_element_by_xpath(CLASS)
                website_text = website_text_element.text
            except Exception as e:
                website_text = np.nan
                print(e)
            companyWebsite_list.append(website_text)

            try:
                industry_element = driver.find_element_by_css_selector('[data-test-id="about-us__industry"]')
                industry_text_element = industry_element.find_element_by_xpath(CLASS)
                industry_text = industry_text_element.text
            except Exception as e:
                industry_text = np.nan
                print(e)
            industry_list.append(industry_text)
            
            try:
                headquarters_element = driver.find_element_by_css_selector('[data-test-id="about-us__headquarters"]')
                headquarters_text_element = headquarters_element.find_element_by_xpath(CLASS)
                headquarters_text = headquarters_text_element.text

                country = location_to_country(headquarters_text)
            except Exception as e:
                headquarters_text = np.nan
                country = np.nan
                print(e)
            headquarter_list.append(headquarters_text)
            country_list.append(country)
            print(country)

        else:
            companyWebsite_list.append(np.nan)
            industry_list.append(np.nan)
            headquarter_list.append(np.nan)
            country_list.append(np.nan)
    except Exception as e:
        companyWebsite_list.append(np.nan)
        industry_list.append(np.nan)
        headquarter_list.append(np.nan)
        country_list.append(np.nan)
    
# driver.quit()

France
Eesti
India
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-test-id="about-us__website"]"}
  (Session info: chrome=117.0.5938.132)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-test-id="about-us__industry"]"}
  (Session info: chrome=117.0.5938.132)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-test-id="about-us__headquarters"]"}
  (Session info: chrome=117.0.5938.132)

nan
France
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-test-id="about-us__website"]"}
  (Session info: chrome=117.0.5938.132)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-test-id="about-us__headquarters"]"}
  (Session info: chrome=117.0.5938.132)

nan
United States
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-test-id="ab

In [42]:
df["Company Website"] = companyWebsite_list
df["Industry"] = industry_list
df["Location"] = headquarter_list
df["Country"] = country_list
df

,Company,Company Description,Date Victimised,Number of Clicks,Company Website,Industry,Location,Country
0,Somagic,The Somagic is a French company in the product...,2023-10-03 14:45:58,337,https://www.somagic.fr,Consumer Goods,"La Genète, Saône-et-Loire",France
1,Windak,Windak is a cable packaging equipment manufact...,2023-10-02 08:19:08,651,http://www.windakgroup.com,Industrial Automation,"Peetri, Harjumaa",Eesti
2,Karam Chand Thapar & Bros Coal Sales,Karam Chand Thapar & Bros Coal Sales - it is t...,2023-10-02 07:57:12,623,http://www.kctgroup.com,"Transportation, Logistics, Supply Chain and St...","Kolkata, West Bengal",India
3,LANDSTAR POWER ONTARIO INC,LANDSTAR POWER ONTARIO INC - canadian company ...,2023-09-26 23:14:54,2070,NaN,NaN,NaN,NaN
4,Acoustic Center,Acoustic Center is an Italian regional company...,2023-09-25 23:12:15,1959,http://www.acoustic-center.com,Retail,PARIS,France
...,...,...,...,...,...,...,...,...
96,European Window,European Window Company is an Australian leade...,2023-02-02 18:24:06,10292,http://www.eurowindow.com.au,Wholesale Building Materials,"Melboune, Victoria",NaN
97,EightPixelsSquare,"Founded in December 2012 and based in Derby, U...",2023-01-25 18:01:43,11011,NaN,NaN,NaN,NaN
98,Aglobis,Resistance shall not prevent us from fulfillin...,2023-01-20 17:59:08,10374,NaN,NaN,NaN,NaN
99,Integerity Tax,"Small business accounting, tax preparation, bo...",2023-01-11 13:11:48,9928,http://www.integritytaxlb.com,Accounting,"Long Beach, California",United States


In [43]:
df.to_csv("Medusa_Linkedin_Data.csv")